
# State-Space Variance Decomposition

This appendix notebook summarises the variance decomposition of the arbitrage spread into structural mean and deviation components using strategy 3 state-space outputs. Inputs are `_output/strategy3/variance_decomposition.csv`, `_output/strategy3/halflife_summary.csv`, and `_output/strategy3/state_estimates.csv`. Outputs include `tables/state_space_variance.csv` and `reports/strategy3_state_space.html`.



## Imports and file paths


In [ ]:

import pathlib

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
})

VAR_PATH = pathlib.Path('_output/strategy3/variance_decomposition.csv')
HALF_PATH = pathlib.Path('_output/strategy3/halflife_summary.csv')
STATE_PATH = pathlib.Path('_output/strategy3/state_estimates.csv')
OUTPUT_TABLE = pathlib.Path('tables/state_space_variance.csv')
OUTPUT_HTML = pathlib.Path('reports/strategy3_state_space.html')
OUTPUT_TABLE.parent.mkdir(parents=True, exist_ok=True)
OUTPUT_HTML.parent.mkdir(parents=True, exist_ok=True)



## Load decomposition outputs


In [ ]:

var_decomp = pd.read_csv(VAR_PATH, comment='#')
half = pd.read_csv(HALF_PATH, comment='#')
state = pd.read_csv(STATE_PATH, comment='#', parse_dates=['date'])
var_decomp['tenor'] = var_decomp['tenor'].astype(int)
half['tenor'] = half['tenor'].astype(int)



## Construct summary table

We consolidate variance shares and half-life metrics into a tidy table, focusing on the structural (μ_t) contribution.


In [ ]:

half_mu = half[half['model'] == 'State Space'].copy() if 'State Space' in half['model'].unique() else half.copy()
half_mu = half_mu[['tenor', 'regime_id', 'half_life_days']]
summary = var_decomp.merge(half_mu, on='tenor', how='left')
summary = summary.rename(columns={'structural_share': 'mu_share', 'deviation_share': 'epsilon_share'})
summary



## Multi-panel visualisation

Panel A plots the variance share attributed to μ_t, while Panel B reports the half-life distribution across regimes.


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
sns.barplot(data=summary, x='tenor', y='mu_share', ax=axes[0], color='#1f77b4')
axes[0].set_title('Panel A: Variance share of μ_t')
axes[0].set_ylabel('Share')
axes[0].set_xlabel('Tenor (years)')
axes[0].set_ylim(0, 1)

sns.barplot(data=half, x='tenor', y='half_life_days', hue='regime_id', ax=axes[1])
axes[1].set_title('Panel B: Half-life by regime')
axes[1].set_ylabel('Half-life (days)')
axes[1].set_xlabel('Tenor (years)')
axes[1].legend(title='Regime')
fig.suptitle('State-space variance and persistence diagnostics')
fig.tight_layout()



## Export table and HTML report


In [ ]:

summary.to_csv(OUTPUT_TABLE, index=False)
html_doc = f'''
<html>
<head><title>State-space variance decomposition</title></head>
<body>
<h1>State-space variance decomposition</h1>
<p>Inputs: {VAR_PATH}, {HALF_PATH}</p>
{summary.to_html(index=False, float_format='{:.4f}'.format)}
</body>
</html>
'''
OUTPUT_HTML.write_text(html_doc)
OUTPUT_HTML



## Interpretation

Across tenors, more than 98% of the spread variance is captured by the structural mean, underscoring the dominance of slow-moving liquidity premia. Half-life bars reveal longer persistence in the slow regime, especially at longer maturities, consistent with the valuation wedge analysis.
